In [10]:
# Compare Silhouette coefficients of different clustering algorithms at different number of clusters
''' This program calculates Silhouette coefficients of different clustering algorithms at different number of clusters. The data 
is Hoechst 33342 fluorescent intensity measured at single-cell level with E.coli. The purpose of clustering in this
experiment is to distinguish 2 subpopulations of bacteria when exposed to Hoechst  '''

import pandas as pd
import numpy as np
import wget
import plotly.express as px
import joblib
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import BayesianGaussianMixture

# Function to load data
def load(url):
    df = pd.read_csv(wget.download(url))
    return df

# Function to subset data
def subset(df,gen):
    df1 = df[["Genotype","Repeat","HT_Conc_uM","Time_m","normed_intensity_ch2"]]
    df1 = df1[df1["Genotype"] == gen] [df1["Time_m"]>0].sort_values(by = ["HT_Conc_uM","Time_m"])
    return df1

# Function to reshape data
def dat(df):
    dat = np.asarray(df['normed_intensity_ch2'],df['normed_intensity_ch2']).reshape(df['normed_intensity_ch2'].size, -1)
    return dat

# Function to calculate range of cluster number needed for screening
def ncluster(df):
    time =  len(df['Time_m'].unique())
    conc = len(df['HT_Conc_uM'].unique())
    r = time*conc # r is the number of clusters needed for screening = number of timepoints * number of HT concentrations
    return r

# Function to calculate Silhouette coefficients for each number of clusters with K-means algorithm
def ktest(df):
    data = dat(df)
    k = []
    n  = []
    for i in range(2,5):
        kmeans = KMeans(n_clusters = i)
        klabels  = kmeans.fit_predict(data)
        k = np.append(k,silhouette_score(data, klabels))
        n = np.append(n,i)
    results = pd.DataFrame(k,n )
    plot = px.scatter(results,
                              title = "Silhouette coeffcicient as a function of cluster number with K-means clustering",
                              template="plotly_dark")
    plot.update_layout(showlegend=False)
    plot.update_xaxes(title = "n cluster")
    plot.update_yaxes(title = "Silhouette coeffcient" )
    plot.show()
    print(f"Best Silhouette coefficient for k-means is {results.max().values}\n")
    print(f"With {results.idxmax().values} clusters")
    return results

# Function to calculate Silhouette coefficients for each number of clusters with Agglomerative clustering algorithm
def aggtest(df):
    data = dat(df)
    k = []
    n  = []
    for i in range(2,5):
        aggclust = AgglomerativeClustering(n_clusters = i,linkage = 'complete')
        klabels  = aggclust.fit_predict(data)
        k = np.append(k,silhouette_score(data, klabels))
        n = np.append(n,i)
    results = pd.DataFrame(k,n )
    plot = px.scatter(results,
                              title = "Silhouette coeffcicient as a function of cluster number with Agglomerative clustering",
                              template="plotly_dark")
    plot.update_layout(showlegend=False)
    plot.update_xaxes(title = "n cluster")
    plot.update_yaxes(title = "Silhouette coeffcient" )
    plot.show()
    print(f"Best Silhouette coefficient for agglomerative clustering is {results.max().values}\n")
    print(f"With {results.idxmax().values} clusters")
    return results

# Calculate Silhouette coefficient for Mean Shift algorithm
def mtest(df):
    data = dat(df)
    k = []
    mshift = MeanShift()
    klabels  = mshift.fit_predict(data)
    k = np.append(k,silhouette_score(data, klabels))
    print(f"Silhoutte coeffecient for Mean shift clustering is{k}\n")
    print(f"With cluster centers:{mshift.fit(data).cluster_centers_} ")
    return k

# Calculate Silhouette coefficient for Bayesian Gaussian Mixture algo
def bayestest(df):
    data = dat(df)
    k = []
    n  = []
    for i in range(2,5):
        bayesmm = BayesianGaussianMixture(n_components = i)
        klabels  = bayesmm.fit_predict(data)
        k = np.append(k,silhouette_score(data, klabels))
        n = np.append(n,i)
    results = pd.DataFrame(k,n )
    plot = px.scatter(results,
                              title = "Silhouette coeffcicient as a function of cluster number with Bayesian Gaussian mixture ",
                              template="plotly_dark")
    plot.update_layout(showlegend=False)
    plot.update_xaxes(title = "n cluster")
    plot.update_yaxes(title = "Silhouette coeffcient" )
    plot.show()
    print(f"Best Silhouette coefficient for Bayesian Gaussian Mixture is {results.max().values}\n")
    print(f"With {results.idxmax().values} clusters")
    return results

if __name__ == "__main__":
    df_acc = load("https://www.dropbox.com/s/di568r38yhduavw/HT_accumulation_demo.csv?dl=1")
    df_accWT = subset(df_acc, "WT")
    df_accTolC = subset(df_acc,"DTolC")
    with joblib.parallel_backend('loky'):
        mwt = mtest(df_accWT)
        kwt = ktest(df_accWT)
        hwt = aggtest(df_accWT)
        bwt = bayestest(df_accWT)
        mtolc = mtest(df_accTolC)
        ktolc = ktest(df_accTolC)
        htolc = aggtest(df_accTolC)
        btolc = bayestest(df_accTolC)

#From the results, it is clear that Agglomerative clustering is the best performer with coefficient ~0.8 at 2 clusters

C:\Users\daile\AppData\Local\Temp\ipykernel_12296\619882243.py:19: DtypeWarning:

Columns (30,33) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\daile\AppData\Local\Temp\ipykernel_12296\619882243.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\daile\AppData\Local\Temp\ipykernel_12296\619882243.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Silhoutte coeffecient for Mean shift clustering is[0.73288475]

With cluster centers:[[  32.37770015]
 [ 208.31639845]
 [ 884.59700855]
 [1607.2025    ]] 


Best Silhouette coefficient for k-means is [0.7694516]

With [3.] clusters


Best Silhouette coefficient for agglomerative clustering is [0.78614016]

With [2.] clusters


c:\Users\daile\Anaconda3\envs\Dropbox\lib\site-packages\sklearn\mixture\_base.py:286: ConvergenceWarning:

Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.



Best Silhouette coefficient for Bayesian Gaussian Mixture is [0.69121481]

With [3.] clusters
Silhoutte coeffecient for Mean shift clustering is[0.67994771]

With cluster centers:[[ 208.18157298]
 [4104.36      ]
 [3363.25      ]
 [2364.82      ]] 


Best Silhouette coefficient for k-means is [0.63682145]

With [4.] clusters


Best Silhouette coefficient for agglomerative clustering is [0.89653106]

With [2.] clusters


KeyboardInterrupt: 